In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math

In [3]:
class EWAlg:
    def __init__(self, epsilon, k, h, myBids, myValue):
        self.weights = np.ones(k)
        self.payoffs = np.zeros(k)
        self.h = h
        self.k = k
        self.epsilon = epsilon
        self.sumWeights = np.sum(self.weights)
        self.probs = self.weights/self.sumWeights
        self.myValue = myValue
        self.myBids = myBids
        self.myTotalValue = 0

    def getBids(self):
        return self.myBids

    def getValue(self):
        return self.myValue

    def getTotalValue(self):
        return self.myTotalValue

    def getAction(self):
        j = np.random.choice(self.k, 1, p = self.probs)
        return self.myBids[j.item()]
    
    def update(self, payoffs):
        for j in range(len(payoffs)):
            curPayoff = payoffs[j]
            self.payoffs[j] = self.payoffs[j] + curPayoff
            newWeight = (1+self.epsilon)**(self.payoffs[j]/self.h)
            self.weights[j] = newWeight
        self.sumWeights = np.sum(self.weights)
        self.weights = self.weights/self.sumWeights
        self.sumWeights = np.sum(self.weights)
        self.probs = self.weights/self.sumWeights
        return

    def generatePayoffs(self, bidToBeat, winningBid, meWin):
        if meWin:
            self.myTotalValue += self.myValue - winningBid
        payoffs = np.zeros(self.k)
        for count, bid in enumerate(self.myBids):
            if bid >= bidToBeat:
                payoffs[count] = self.myValue - bid
            else:
                payoffs[count] = 0
        return payoffs
    
    def reset(self):
        self.weights = np.ones(self.k)
        self.payoffs = np.zeros(self.k)
        self.sumWeights = np.sum(self.weights)
        self.probs = self.weights/self.sumWeights
        self.myTotalValue = 0


# Part 1 : Online Reserve Pricing

In [30]:
class Auction:
    def __init__(self, numItems, k = 100, h = 1, reserve = 0, learningRate = 0.1):
        self.reserve = reserve
        self.k = k
        self.numItems = numItems
        self.possibleReserves = np.linspace(0, h, k)
        self.learner = EWAlg(learningRate, k, h, self.possibleReserves, h)

    def runAuction(self, bids):
        return self.runAuctionHelper(bids, self.numItems, self.reserve)
    
    def runAuctionHelper(self, bids, numItemsToSell, reservePrice):
        # print("running auction helper. bids are", bids)
        bids.append(reservePrice)
        bids.sort()
        winningBids = []
        if (bids[-(numItemsToSell+1)] > reservePrice):
            salePrice = bids[-(numItemsToSell+1)]
            for i in range(numItemsToSell):
                winningBids.append(bids[-(i+1)])
                # print("winningbid is", winningBids)
            saleRevenue = salePrice * numItemsToSell
        else:
            numItemsToSell -= 1
            # print("looping back!")
            return self.runAuctionHelper(bids, numItemsToSell, reservePrice)
        return winningBids, salePrice, saleRevenue

    def generatePayoffs(self, winningBids, salePrice, itemsSold):
        # calculate the payoff for a reserve valuee
        # repeeat for all possible reserve values
        # print("running generate payoffs")
        payoffs = np.zeros(self.k)
        for count, reserve in enumerate(self.possibleReserves):
            # print("running auction helper in generate payoffs")
            bids, sp, saleRevenue = self.runAuctionHelper(winningBids, itemsSold, reserve)
            payoffs[count] = saleRevenue
        return payoffs

    
    def updateReservePrice(self, winningBids, salePrice):
        payoffs = self.generatePayoffs(winningBids, salePrice, len(winningBids))
        self.learner.update(payoffs)
        return


In [34]:
auction = Auction(1, k = 100, h = 1, reserve = 0, learningRate = 0.1)
bids = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 10]
winningBids, salePrice, saleRevenue = auction.runAuction(bids)
auction.updateReservePrice(winningBids, salePrice)
print("winningbids is", winningBids)
print(salePrice)
print(saleRevenue)

winningbids is [0.0, 0.0, 0.010101010101010102, 0.010101010101010102, 0.020202020202020204, 0.020202020202020204, 0.030303030303030304, 0.030303030303030304, 0.04040404040404041, 0.04040404040404041, 0.05050505050505051, 0.05050505050505051, 0.06060606060606061, 0.06060606060606061, 0.07070707070707072, 0.07070707070707072, 0.08080808080808081, 0.08080808080808081, 0.09090909090909091, 0.09090909090909091, 0.10101010101010102, 0.10101010101010102, 0.11111111111111112, 0.11111111111111112, 0.12121212121212122, 0.12121212121212122, 0.13131313131313133, 0.13131313131313133, 0.14141414141414144, 0.14141414141414144, 0.15151515151515152, 0.15151515151515152, 0.16161616161616163, 0.16161616161616163, 0.17171717171717174, 0.17171717171717174, 0.18181818181818182, 0.18181818181818182, 0.19191919191919193, 0.19191919191919193, 0.20202020202020204, 0.20202020202020204, 0.21212121212121213, 0.21212121212121213, 0.22222222222222224, 0.22222222222222224, 0.23232323232323235, 0.23232323232323235, 0.